# Libraries

In [3]:
%matplotlib inline
import cv2
import os

import numpy as np
import mediapipe as mp

from matplotlib import pyplot as plt

# Import mediapipe model for finger and pose detection

In [4]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

# Functions

In [5]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [6]:
def draw_landmarks(image, results):
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [7]:
def draw_styled_landmarks(image, results):
    # # Draw face connections
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
    #                          mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
    #                          mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    #                          ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [8]:
def extract_keypoints(results):
    # pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([face, lh, rh])

In [9]:
def extract_coordinates(filepath, savepath):
    cap = cv2.VideoCapture(filepath)
    frame_num = 1
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        # while(frame_num < 31):
        while cap.isOpened():
            ret, frame = cap.read()

            if ret == True:
            
                image, results = mediapipe_detection(frame, holistic)
                draw_styled_landmarks(image, results)
                keypoints = extract_keypoints(results)

                save_dest = os.path.join(savepath, str(frame_num))

                np.save(save_dest, keypoints)

                frame_num += 1
            
                cv2.imshow('frame', image)
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break
 
            else:
                break
    
    cap.release()
    cv2.destroyAllWindows()

    return frame_num

In [10]:
def count_files_in_dir(directory_path):
    count = 0
    for path in os.listdir(directory_path):
        if os.path.isfile(os.path.join(directory_path, path)):
            count += 1
    return count

# Folder creation

Create folders for
1. Label folders for new actions
2. Sub-label folders

In [11]:
# set array of actions, may be updated when more data samples are added
actions = np.array(sorted(['door', 'house', 'again', 'open'])) # sorted to follow folder arrangement

# current directory
c_dir = os.getcwd()

# Keypoint extraction
Here we extract the keypoints of the data (videos) by looping through each set of videos in each action folder, then writing the corresponding keypoints. 

This will be used later for model training.

In [27]:
video_count = 0 # required for processing later
for action in actions:
    counter = 0
    for video in os.listdir(os.path.join('videos', action)):
        counter += 1
        video_count += 1
        filepath = os.path.join(os.path.join(c_dir, 'videos', action, video))

        # NOTE: for each video, save keypoints in new subfolder
        # create subfolder if it does not exist
        subfolder = str(action) + '_' + str(counter)
        if not os.path.isdir(os.path.join(c_dir, 'labels', action, subfolder)):
            os.mkdir(os.path.join(c_dir, 'labels', action, subfolder))
        # set new savepath
        savepath = os.path.join(c_dir, 'labels', action, subfolder)
        print(f'Currently reading video {video_count}: {filepath}')
        extract_coordinates(filepath, savepath)


Currently reading video 1: d:\GitHub\SSLrecognition\train_data\videos\again\fq_again_1.mp4


c:\Users\wei-z\anaconda3\envs\is460proj\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Currently reading video 2: d:\GitHub\SSLrecognition\train_data\videos\again\fq_again_2.mp4
Currently reading video 3: d:\GitHub\SSLrecognition\train_data\videos\door\fq_door_1.mp4
Currently reading video 4: d:\GitHub\SSLrecognition\train_data\videos\house\fq_house_1.mp4
Currently reading video 5: d:\GitHub\SSLrecognition\train_data\videos\house\zh_house_1.mp4
Currently reading video 6: d:\GitHub\SSLrecognition\train_data\videos\house\zh_house_2.mp4
Currently reading video 7: d:\GitHub\SSLrecognition\train_data\videos\house\zh_house_3.mp4
Currently reading video 8: d:\GitHub\SSLrecognition\train_data\videos\open\fq_open_1.mp4


# Model training

In [12]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras_preprocessing.sequence import pad_sequences

In [13]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'again': 0, 'door': 1, 'house': 2, 'open': 3}

In [14]:
sequences, labels = [], []
for action in actions:
    no_actions = len(os.listdir(os.path.join(c_dir, 'labels', action)))
    print('Opening path:', os.path.join(c_dir, 'labels', action))
    print(f'Number of instances: {no_actions}')
    for num in range(1, no_actions + 1):
        window = []
        file = str(action) + "_" + str(num)
        no_frames_per_action = len(os.listdir(os.path.join(c_dir, 'labels', action, file)))
        print(f'Number of frames in {file}: {no_frames_per_action}')
        for frame_num in range(1, no_frames_per_action + 1):
            res = np.load(os.path.join(c_dir, 'labels', action, file,  "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
    print('-'*75)

Opening path: d:\GitHub\SSLrecognition\train_data\labels\again
Number of instances: 2
Number of frames in again_1: 96
Number of frames in again_2: 96
---------------------------------------------------------------------------
Opening path: d:\GitHub\SSLrecognition\train_data\labels\door
Number of instances: 1
Number of frames in door_1: 87
---------------------------------------------------------------------------
Opening path: d:\GitHub\SSLrecognition\train_data\labels\house
Number of instances: 4
Number of frames in house_1: 105
Number of frames in house_2: 57
Number of frames in house_3: 59
Number of frames in house_4: 67
---------------------------------------------------------------------------
Opening path: d:\GitHub\SSLrecognition\train_data\labels\open
Number of instances: 1
Number of frames in open_1: 117
---------------------------------------------------------------------------


In [15]:
x = np.array(pad_sequences(sequences, dtype = 'float', padding = 'post', value = 0))
y = pad_sequences(to_categorical(labels).astype(int), dtype = 'int', padding = 'post', value = -1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)

# Models

## LSTM

In [46]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [52]:
log_dir = os.path.join(c_dir, 'Logs')
tb_callback = TensorBoard(log_dir = log_dir)

In [53]:
model = Sequential()
model.add(LSTM(64, return_sequences = True, activation = "relu", input_shape = (117, 1530)))
model.add(LSTM(128, return_sequences = True, activation = "relu"))
model.add(LSTM(64, return_sequences = False, activation = "relu"))
model.add(Dense(64, activation = "relu"))
model.add(Dense(32, activation = "relu"))
model.add(Dense(actions.shape[0], activation = "softmax"))

In [54]:
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ['categorical_accuracy'])

In [55]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_16 (LSTM)              (None, 117, 64)           408320    
                                                                 
 lstm_17 (LSTM)              (None, 117, 128)          98816     
                                                                 
 lstm_18 (LSTM)              (None, 64)                49408     
                                                                 
 dense_15 (Dense)            (None, 64)                4160      
                                                                 
 dense_16 (Dense)            (None, 32)                2080      
                                                                 
 dense_17 (Dense)            (None, 4)                 132       
                                                                 
Total params: 562916 (2.15 MB)
Trainable params: 56291

In [64]:
model.fit(x_train, y_train, epochs = 2000, callbacks = [tb_callback])

Epoch 1/2000
1/1 [==============================] - 0s 149ms/step - loss: 14024.1250 - categorical_accuracy: 0.4286
Epoch 2/2000
1/1 [==============================] - 0s 111ms/step - loss: 7416.7856 - categorical_accuracy: 0.1429
Epoch 3/2000
1/1 [==============================] - 0s 110ms/step - loss: 7578.2144 - categorical_accuracy: 0.5714
Epoch 4/2000
1/1 [==============================] - 0s 112ms/step - loss: 7958.6431 - categorical_accuracy: 0.5714
Epoch 5/2000
1/1 [==============================] - 0s 109ms/step - loss: 7224.9644 - categorical_accuracy: 0.5714
Epoch 6/2000
1/1 [==============================] - 0s 108ms/step - loss: 10388.2139 - categorical_accuracy: 0.4286
Epoch 7/2000
1/1 [==============================] - 0s 107ms/step - loss: 3269.0715 - categorical_accuracy: 0.2857
Epoch 8/2000
1/1 [==============================] - 0s 111ms/step - loss: 10177.3213 - categorical_accuracy: 0.2857
Epoch 9/2000
1/1 [==============================] - 0s 113ms/step - loss: 620

In [63]:
res = model.predict(x_test)
print(res)
print(y_test)

1/1 [==============================] - 0s 21ms/step
[[0. 0. 1. 0.]]
[[0 0 1 0]]
